In [2]:
import json
from datetime import timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

import torch
from chronos import ChronosPipeline

In [3]:
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-tiny",
  device_map="mps",
  torch_dtype=torch.bfloat16,
)

In [5]:
# load data

PATH = "data"

sales_df = pd.read_csv(f"{PATH}/shop_sales.csv")
dates_df = pd.read_csv(f"{PATH}/shop_sales_dates.csv")

# preprocces
train = sales_df.merge(dates_df[['date', 'date_id', 'year', 'wm_yr_wk']], on='date_id')
# del dates_df, sales_df

train['store_item_id'] = train.item_id
train.item_id = train.item_id.apply(lambda x: x.split('_')[-1])

train.date = pd.to_datetime(train.date)
train = train.rename(columns={'cnt': 'sales'})

In [6]:
train

,item_id,store_id,date_id,sales,date,year,wm_yr_wk,store_item_id
0,085,STORE_2,1,3,2011-01-29,2011,11101,STORE_2_085
1,085,STORE_2,2,8,2011-01-30,2011,11101,STORE_2_085
2,085,STORE_2,3,0,2011-01-31,2011,11101,STORE_2_085
3,085,STORE_2,4,3,2011-02-01,2011,11101,STORE_2_085
4,085,STORE_2,5,0,2011-02-02,2011,11101,STORE_2_085
...,...,...,...,...,...,...,...,...
81850,727,STORE_1,1815,2,2016-01-17,2016,11551,STORE_1_727
81851,727,STORE_1,1816,3,2016-01-18,2016,11551,STORE_1_727
81852,727,STORE_1,1817,1,2016-01-19,2016,11551,STORE_1_727
81853,727,STORE_1,1818,4,2016-01-20,2016,11551,STORE_1_727


In [ ]:
df = train[(train.store_nbr == store) & (train.family == family)]
context = torch.tensor(df["sales"].tolist())
embeddings, tokenizer_state = pipeline.embed(context)
        

# result = {
#     'date': df.date.max().strftime("%Y-%m-%d"),
#     'embedding': embeddings[0].mean(axis=0).float().numpy().tolist(),
#     'tokenizer_state': float(tokenizer_state)
# }

float(tokenizer_state)

embeddings[0].mean(axis=0).float().numpy().tolist()

In [10]:
for store in tqdm(train.store_nbr.unique()):
    for family in train.family.unique():
        all_embeddings = []
        
        df = train[(train.store_nbr == store) & (train.family == family)]
        context = torch.tensor(df["sales"].tolist())
        embeddings, tokenizer_state = pipeline.embed(context)
        
        result = {
            'store_nbr': int(store),
            'family': family,
            'date': df.date.max().strftime("%Y-%m-%d"),
            'embedding': embeddings[0].mean(axis=0).float().numpy().tolist(),
            'tokenizer_state': float(tokenizer_state)
        }
        all_embeddings.append(result)

#         for i in range(1, 365):
#             sales = df["sales"].tolist()[:-i]
#             if len(sales) < 10:
#                 break
#             context = torch.tensor(sales)
#             embeddings, tokenizer_state = pipeline.embed(context)
            
#             result = {
#                 'store_nbr': int(store),
#                 'family': family,
#                 'date': (df.date.max() - timedelta(days=i)).strftime("%Y-%m-%d"),
#                 'embedding': embeddings[0].mean(axis=0).float().numpy().tolist(),
#                 'tokenizer_state': float(tokenizer_state)
#             }
#             all_embeddings.append(result)

        # json.dump(all_embeddings, open(f'embeddings/store_category/{store}_{family.replace("/", "_")}.json', 'w'))


  0%|          | 0/54 [00:00<?, ?it/s]


KeyboardInterrupt

